In [9]:
from casadi import SX, inv, vertcat, Function, integrator, sin, cos

In [13]:
#Equations of motion expressed in BODY
m = SX.sym('m')  # Mass

X_du = SX.sym('X_du') # Added mass in surge
Y_dv = SX.sym('Y_dv') # Added mass in sway
Z_dw = SX.sym('Z_dw') # Added mass in heave

K_dp = SX.sym('K_dp') # Added mass in roll
M_dq = SX.sym('M_dq') # Added mass in pitch
N_dr = SX.sym('N_dr') # Added mass in yaw

X_dw = SX.sym('X_dw') # coupled Added mass in surge & heave
Y_dp = SX.sym('Y_dp') # coupled Added mass in sway & roll
X_dq = SX.sym('X_dq') # coupled Added mass in surge & pitch
Y_dr = SX.sym('Y_dr') # coupled Added mass in sway & yaw
Z_dq = SX.sym('Z_dq') # coupled Added mass in heave & pitch
K_dr = SX.sym('K_dr') # coupled Added mass in roll & yaw

I_x = SX.sym('I_x') # moment of inertia x entry
I_y = SX.sym('I_y') # moment of inertia y entry
I_z = SX.sym('I_z') # moment of inertia z entry
I_zx = SX.sym('I_zx') # product of inertia zx entry 

x_g = SX.sym('x_g')  # Center of gravity, x-axis
y_g = SX.sym('y_g')  # Center of gravity, z-axis
z_g = SX.sym('z_g')  # Center of gravity, z-axis

x_b = SX.sym('x_b')  # Center of buoyancy, x-axis
y_b = SX.sym('y_b')  # Center of buoyancy, z-axis
z_b = SX.sym('z_b')  # Center of buoyancy, z-axis

In [12]:
# Inertia Matrix ROV & AUV
#assuming Starboard–port symmetrical underwater vehicles yg=0 and Ixy = Iyz = 0
M = SX(6, 6)  # Create a 6x6 symbolic matrix

# Manually populate the matrix
M[0, 0] = m - X_du
M[1, 1] = m - Y_dv
M[2, 2] = m - Z_dw
M[3, 3] = I_x - K_dp
M[4, 4] = I_y - M_dq
M[5, 5] = I_z - N_dr

M[2, 0] = -X_dw
M[0, 2] = -X_dw

M[3, 1] = -m * z_g - Y_dp
M[1, 3] = -m * z_g - Y_dp

M[4, 2] = m * x_g - Z_dq
M[2, 4] = m * x_g - Z_dq

M[5, 3] = -I_zx - K_dr
M[5, 3] = -I_zx - K_dr

M[4, 0] = m*z_g - X_dq
M[0, 4] = m*z_g - X_dq

M[5, 1] = m * x_g - Y_dr
M[1, 5] = m * x_g - Y_dr

In [16]:
# restoring forces

W = SX.sym('W')  # weight
B = SX.sym('B')  # buoyancy

thet = SX.sym('thet')  
phi = SX.sym('phi')  

g = SX(6, 1)

g[0,0] = (W - B)*sin(thet)
g[1,0] = -(W - B)*cos(thet)*sin(phi)
g[2,0] = -(W - B)*cos(thet)*cos(phi)
g[3,0] = -(y_g*W -y_b*B)*cos(thet)*cos(phi) + (z_g*W -z_b*B)*cos(thet)*sin(phi)
g[4,0] = (z_g*W -z_b*B)*sin(thet) + (x_g*W -x_b*B)*cos(thet)*cos(phi)
g[5,0] = -(x_g*W -x_b*B)*cos(thet)*sin(phi) - (y_g*W -y_b*B)*sin(thet)

In [ ]:
# Damping will be nonlinear and coupled for an underwater vehicle moving in 6 DOFs at high speed

# <xUabsU>	Quadratic drag in surge

# <yVabsV>	Quadratic drag in sway

# <zWabsW>	Quadratic drag in heave

# <kPabsP>	Quadratic drag in roll

# <mQabsQ>	Quadratic drag in pitch

# <nRabsR>	Quadratic drag in yaw



# vehicle is performing a noncoupled motion, we can assume a diagonal structure
X_u = SX.sym('X_u') # <xU>	Linear drag in surge
Y_v = SX.sym('Y_v') # <yV>	Linear drag in sway
Z_w = SX.sym('Z_w') # <zW>	Linear drag in heave
K_p = SX.sym('K_p') # <kP>	Linear drag in roll
M_q = SX.sym('M_q') # <mQ>	Linear drag in pitch
N_r = SX.sym('N_r') # <nR>	Linear drag in yaw

